---
# **Estratégia Fractal**
---

# *Tasks*

* Identificar os halts na MM
* Definir as entradas pelo timeframe anterior

## 1. Bibliotecas

In [19]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import pytz

import MetaTrader5 as mt5

import plotly.graph_objects as go

## 2. Obtenção de dados

In [20]:
mt5.initialize()

True

In [94]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=1500)
data_fim = datetime.now()

In [124]:
ohlc_1min = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_M1, data_inicio, data_fim)
ohlc_1min = pd.DataFrame(ohlc_1min)
ohlc_1min['mm_20p'] = ohlc_1min.close.rolling(20).mean()
ohlc_1min.index = pd.to_datetime(ohlc_1min['time'], unit='s')
ohlc_1min = ohlc_1min.drop('time', axis=1)

ohlc_5min = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_M5, data_inicio, data_fim)
ohlc_5min = pd.DataFrame(ohlc_5min)
ohlc_5min['mm_20p'] = ohlc_5min.close.rolling(20).mean()
ohlc_5min.index = pd.to_datetime(ohlc_5min['time'], unit='s')
ohlc_5min = ohlc_5min.drop('time', axis=1)

ohlc_15min = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_M15, data_inicio, data_fim)
ohlc_15min = pd.DataFrame(ohlc_15min)
ohlc_15min['mm_20p'] = ohlc_15min.close.rolling(20).mean()
ohlc_15min.index = pd.to_datetime(ohlc_15min['time'], unit='s')
ohlc_15min = ohlc_15min.drop('time', axis=1)

ohlc_60min = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_H1, data_inicio, data_fim)
ohlc_60min = pd.DataFrame(ohlc_60min)
ohlc_60min['mm_20p'] = ohlc_60min.close.rolling(20).mean()
ohlc_60min.index = pd.to_datetime(ohlc_60min['time'], unit='s')
ohlc_60min = ohlc_60min.drop('time', axis=1)

ohlc_1dia = mt5.copy_rates_range('WIN$', mt5.TIMEFRAME_D1, data_inicio, data_fim)
ohlc_1dia = pd.DataFrame(ohlc_1dia)
ohlc_1dia['mm_20p'] = ohlc_1dia.close.rolling(20).mean()
ohlc_1dia['bb_up_mm_20p'] =  ohlc_1dia.close.rolling(20).mean() + 0.5*ohlc_1dia.close.rolling(20).std()
ohlc_1dia['bb_down_mm_20p'] =  ohlc_1dia.close.rolling(20).mean() - 0.5*ohlc_1dia.close.rolling(20).std()
ohlc_1dia['mm_13p'] =  ohlc_1dia.close.rolling(13).mean()
ohlc_1dia.index = pd.to_datetime(ohlc_1dia['time'], unit='s')
ohlc_1dia = ohlc_1dia.drop('time', axis=1)

# **INCLUIR MÉDIAS EXPONENCIAIS**

In [102]:
ohlc_1dia.head()

,open,high,low,close,tick_volume,spread,real_volume,mm_20p,bb_up_mm_20p,bb_down_mm_20p
time,,,,,,,,,,
2018-10-15,105377.0,105690.0,103814.0,103814.0,1028432,1,3343609,NaN,NaN,NaN
2018-10-16,104764.0,107534.0,104527.0,107384.0,1011620,1,3354213,NaN,NaN,NaN
2018-10-17,106974.0,107842.0,106163.0,107222.0,1108292,1,3653555,NaN,NaN,NaN
2018-10-18,106850.0,107080.0,104477.0,104743.0,1038793,1,3470886,NaN,NaN,NaN
2018-10-19,105729.0,106206.0,104508.0,105481.0,1031990,1,3480218,NaN,NaN,NaN


## 3. Identificar os halts na MM

### 3.1. Halt de alta

In [109]:
ohlc_1dia['halt_alta'] =  (ohlc_1dia.low <= ohlc_1dia.mm_20p) & (ohlc_1dia.high > ohlc_1dia.mm_20p) & \
    (ohlc_1dia.low.shift(1) > ohlc_1dia.mm_20p.shift(1)) & (ohlc_1dia.high.shift(1) > ohlc_1dia.mm_20p.shift(1))

In [110]:
halts_mm = ohlc_1dia[ohlc_1dia['halt_alta']==True]
halts_mm.head(10)

,open,high,low,close,tick_volume,spread,real_volume,mm_20p,bb_up_mm_20p,bb_down_mm_20p,mm_13p,halt_alta
time,,,,,,,,,,,,
2018-11-21,107836.0,108710.0,107303.0,108524.0,1183446,1,3626465,107650.10,109799.914286,105500.285714,108541.230769,True
2018-11-23,108443.0,108567.0,106683.0,107024.0,1051039,1,3416035,107748.35,109899.804203,105596.895797,108338.153846,True
2018-12-06,108685.0,110290.0,107892.0,110247.0,1260674,1,4422413,108813.75,110649.169581,106978.330419,109498.923077,True
2018-12-10,109082.0,109528.0,106417.0,106417.0,1473607,1,4772362,108778.40,110620.812989,106935.987011,109318.692308,True
2019-02-06,120603.0,120670.0,116235.0,116506.0,1737839,1,6382476,118174.70,120021.000913,116328.399087,119106.538462,True
2019-02-18,119200.0,119382.0,118159.0,118537.0,1428989,1,5033112,118688.85,120356.333268,117021.366732,118729.076923,True
2019-02-26,119029.0,120143.0,118750.0,119577.0,1526840,1,5438820,118965.90,120539.040436,117392.759564,118767.846154,True
2019-03-20,121579.0,122005.0,119029.0,119029.0,1772371,1,6120616,119036.50,121015.381117,117057.618883,118861.384615,True
2019-04-09,118214.0,118384.0,116321.0,117313.0,1718917,1,6089032,117501.10,120459.820917,114542.379083,115905.769231,True


In [111]:
print('Identificados',len(halts_mm),'halts considerando o dataset de',len(ohlc_1dia),'candles')

Identificados 47 halts considerando o dataset de 1017 candles


### 3.1.2 Halt alta com 1std MM

In [125]:
ohlc_1dia['halt_alta'] =  (ohlc_1dia.low <= ohlc_1dia.bb_up_mm_20p) & (ohlc_1dia.high > ohlc_1dia.bb_up_mm_20p) & \
    (ohlc_1dia.low.shift(1) > ohlc_1dia.mm_20p.shift(1)) & (ohlc_1dia.high.shift(1) > ohlc_1dia.mm_20p.shift(1))

In [126]:
halts_mm = ohlc_1dia[ohlc_1dia['halt_alta']==True]
halts_mm.head(10)
print('Identificados',len(halts_mm),'halts considerando o dataset de',len(ohlc_1dia),'candles')

Identificados 97 halts considerando o dataset de 1017 candles


In [90]:
candles_sinais = halts_mm.index

df_figure = ohlc_1dia[ohlc_1dia.index.isin(candles_sinais)]
df_figure

,open,high,low,close,tick_volume,spread,real_volume,mm_20p,halt_alta,MM_ascend_em_tend,mm_ascend_em_tend,toque_mm
time,,,,,,,,,,,,
2021-10-19,127065.0,127076.0,122630.0,124034.0,6454208,1,23257211,125376.50,True,False,False,False
2021-12-20,116762.0,117002.0,115112.0,116054.0,5588653,1,18497332,116965.05,True,False,False,False
2022-02-18,124146.0,124641.0,122752.0,122860.0,5174982,1,16688996,122845.00,True,True,True,True
2022-03-04,124619.0,125044.0,123274.0,124393.0,5042732,1,16046084,123399.10,True,True,True,True
2022-04-06,127509.0,127918.0,125825.0,127213.0,5544941,1,17472999,126208.80,True,False,False,False
2022-04-08,128381.0,128381.0,126470.0,127348.0,4073151,1,14060413,126735.65,True,True,True,True
2022-06-07,116557.0,117107.0,115954.0,116726.0,4672190,1,14861621,116518.00,True,False,False,False
2022-08-30,116391.0,116538.0,113608.0,113633.0,5072881,1,16682466,114919.85,True,False,False,False
2022-09-13,117159.0,117377.0,113486.0,113720.0,5758573,1,17477204,115408.75,True,False,False,False


In [91]:
candles_sinais = halts_mm.index
candles_sinais

DatetimeIndex(['2021-10-19', '2021-12-20', '2022-02-18', '2022-03-04',
               '2022-04-06', '2022-04-08', '2022-06-07', '2022-08-30',
               '2022-09-13', '2022-10-13', '2022-10-25', '2022-11-07'],
              dtype='datetime64[ns]', name='time', freq=None)

In [77]:
candles_sinais[0]

Timestamp('2021-10-19 00:00:00')

## incluir marcaçao vertical ao longo de todos os candles sinais

# ALTERAR NOME PARA EVITAR NUMERO (PENSANDO EM OTIMIZACAO)

## INCLUIOR FILTRO MEDIA MOVEL ASCENDENTE

In [127]:
df_figure = ohlc_1dia

candles_sinais = halts_mm.index
df_figure_sinais = ohlc_1dia[ohlc_1dia.index.isin(candles_sinais)]

fig_halt_alta = go.Figure(data=[go.Candlestick(name='Ativo',x=df_figure.index, open=df_figure['open'], high = df_figure['high'],
                                              low=df_figure['low'], close=df_figure['close'],increasing_line_color= 'green',
                                              decreasing_line_color= 'red', showlegend=True)])

#for i in candles_sinais:
#    fig_halt_alta.add_vrect(x0=i+timedelta(minutes=250), x1=i+timedelta(minutes=250),fillcolor="gray", opacity=0.25)

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.mm_20p, line_width=3, line_dash="dash", line_color="black",
                        name='mm_20p',showlegend=True))

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.bb_up_mm_20p, line_width=3, line_dash="dash", line_color="blue",
                        name='bb_up_mm_20p',showlegend=True))

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.mm_13p, line_width=3, line_dash="dash", line_color="red",
                        name='mm_13p',showlegend=True))

fig_halt_alta.update_xaxes(title_text="<b> Data", rangebreaks=[
                        dict(bounds=["sat", "mon"])])

# Configurações do layout, como fundo branco, tamanho da imagem, margens, titulo do gráfico...
fig_halt_alta.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Halt de alta', template = 'simple_white',
                            margin=dict(l=20, r=20, t=100, b=20), paper_bgcolor="#f7f8fa",width=800,height=600,
                            font=dict(family="Arial",size=17, color = 'black'),
                            legend=dict(orientation="h",yanchor="top",y=1.1,xanchor="right",x=1.05))

In [132]:
index_fig = 0

# Ajuste de acordo com o timeframe
inicio_index = halts_mm.index[index_fig] - timedelta(days=20)
final_index = halts_mm.index[index_fig] + timedelta(days=5)

data_range = pd.date_range(start=inicio_index, end=final_index, freq='B')
df_figure = ohlc_1dia[ohlc_1dia.index.isin(data_range)]

#df_figure = df_figure.iloc[0:15]
candle_sinal = halts_mm.index[index_fig]

fig_halt_alta = go.Figure(data=[go.Candlestick(name='Ativo',x=df_figure.index, open=df_figure['open'], high = df_figure['high'],
                                              low=df_figure['low'], close=df_figure['close'],increasing_line_color= 'green',
                                              decreasing_line_color= 'red', showlegend=True)])

fig_halt_alta.add_vrect(x0=halts_mm.index[index_fig]-timedelta(minutes=120), x1=(halts_mm.index[index_fig]+timedelta(minutes=120)),
                        fillcolor="gray", opacity=0.25)

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.bb_up_mm_20p, line_width=2, line_dash="dash", line_color="lightgray",
                        name='bb_up_mm_20p',showlegend=True))

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.mm_20p, line_width=3, line_dash="dash", line_color="black",
                        name='mm_20p',showlegend=True))

fig_halt_alta.update_xaxes(title_text="<b> Data", rangebreaks=[
                        dict(bounds=["sat", "mon"])])

# Configurações do layout, como fundo branco, tamanho da imagem, margens, titulo do gráfico...
fig_halt_alta.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Halt de alta', template = 'simple_white',
                            margin=dict(l=20, r=20, t=100, b=20), paper_bgcolor="#f7f8fa",width=800,height=600,
                            font=dict(family="Arial",size=17, color = 'black'),
                            legend=dict(orientation="h",yanchor="top",y=1.1,xanchor="right",x=1.05))

### 3.2. Tentativa anterior

MM ascendente será considerada qd MM20 atual > MM20 atrasada em 10p e > MM20p de 10p > 20p 

Objetivo: Nao operar consolidações

In [82]:
ohlc_1dia['mm_ascend_em_tend'] =  (ohlc_1dia.close > ohlc_1dia.mm_20p) & (ohlc_1dia.mm_20p > ohlc_1dia.mm_20p.shift(1)) & \
                                (ohlc_1dia.mm_20p > ohlc_1dia.mm_20p.shift(10)) & (ohlc_1dia.mm_20p.shift(10) > ohlc_1dia.mm_20p.shift(20))

In [83]:
mm_ascend_em_tend = ohlc_1dia[ohlc_1dia['mm_ascend_em_tend']==True]
mm_ascend_em_tend.head(10)

,open,high,low,close,tick_volume,spread,real_volume,mm_20p,halt_alta,MM_ascend_em_tend,mm_ascend_em_tend,toque_mm
time,,,,,,,,,,,,
2022-02-08,121881.0,122807.0,120956.0,122807.0,4840208,1,15910378,120307.35,False,True,True,False
2022-02-09,123080.0,123472.0,121854.0,122421.0,5318717,1,17438058,120615.35,False,True,True,False
2022-02-10,122535.0,124152.0,122257.0,123619.0,6164434,1,19512530,121007.20,False,True,True,False
2022-02-11,123210.0,125247.0,123188.0,123951.0,6461530,1,20210442,121323.90,False,True,True,False
2022-02-14,123080.0,124506.0,122949.0,124125.0,6096582,1,18490020,121702.95,False,True,True,False
2022-02-15,124713.0,125405.0,124147.0,125405.0,4405551,1,13510925,122117.10,False,True,True,False
2022-02-16,125114.0,126207.0,125114.0,125572.0,4118034,1,13658563,122467.75,False,True,True,False
2022-02-17,125163.0,125518.0,123549.0,123656.0,4511191,1,14755035,122668.10,False,True,True,False
2022-02-18,124146.0,124641.0,122752.0,122860.0,5174982,1,16688996,122845.00,True,True,True,True


Proximo passo identificar o toque na média

Sinal será apenas no candle sinalizador ou candle anterior completamente fora da MM

In [85]:
ohlc_1dia['toque_mm'] = (ohlc_1dia.low <= ohlc_1dia.mm_20p) & (ohlc_1dia.mm_ascend_em_tend==True)
toque_mm = ohlc_1dia[ohlc_1dia['toque_mm']==True]
toque_mm

,open,high,low,close,tick_volume,spread,real_volume,mm_20p,halt_alta,MM_ascend_em_tend,mm_ascend_em_tend,toque_mm
time,,,,,,,,,,,,
2022-02-18,124146.0,124641.0,122752.0,122860.0,5174982,1,16688996,122845.00,True,True,True,True
2022-02-22,122311.0,123543.0,122241.0,123543.0,5563051,1,18498273,123128.65,False,True,True,True
2022-03-04,124619.0,125044.0,123274.0,124393.0,5042732,1,16046084,123399.10,True,True,True,True
2022-03-09,122031.0,123904.0,121245.0,123538.0,5117818,1,17287476,123261.90,False,True,True,True
2022-03-10,122849.0,123387.0,121369.0,123333.0,5472458,1,17734707,123288.20,False,True,True,True
2022-04-08,128381.0,128381.0,126470.0,127348.0,4073151,1,14060413,126735.65,True,True,True,True


In [86]:
print('Identificados',len(toque_mm),'halts considerando o dataset de',len(ohlc_1dia),'candles')

Identificados 6 halts considerando o dataset de 342 candles


In [70]:
index_fig = 1

# Ajuste de acordo com o timeframe
inicio_index = toque_mm.index[index_fig] - timedelta(days=10)
final_index = toque_mm.index[index_fig] + timedelta(days=5)

data_range = pd.date_range(start=inicio_index, end=final_index, freq='B')
df_figure = ohlc_1dia[ohlc_1dia.index.isin(data_range)]

#df_figure = df_figure.iloc[0:15]
candle_sinal = toque_mm.index[index_fig]

fig_halt_alta = go.Figure(data=[go.Candlestick(name='Ativo',x=df_figure.index, open=df_figure['open'], high = df_figure['high'],
                                              low=df_figure['low'], close=df_figure['close'],increasing_line_color= 'green',
                                              decreasing_line_color= 'red', showlegend=True)])

fig_halt_alta.add_vrect(x0=toque_mm.index[index_fig]-timedelta(minutes=120), x1=(toque_mm.index[index_fig]+timedelta(minutes=120)),
                        fillcolor="gray", opacity=0.25)

fig_halt_alta.add_trace(go.Scatter(x=df_figure.index, y=df_figure.mm_20p, line_width=3, line_dash="dash", line_color="black",
                        name='mm_20p',showlegend=True))

fig_halt_alta.update_xaxes(title_text="<b> Data", rangebreaks=[
                        dict(bounds=["sat", "mon"])])

# Configurações do layout, como fundo branco, tamanho da imagem, margens, titulo do gráfico...
fig_halt_alta.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Halt de alta', template = 'simple_white',
                            margin=dict(l=20, r=20, t=100, b=20), paper_bgcolor="#f7f8fa",width=800,height=600,
                            font=dict(family="Arial",size=17, color = 'black'),
                            legend=dict(orientation="h",yanchor="top",y=1.1,xanchor="right",x=1.05))